In [ ]:
import typing as tp

# импорты встроенных и third-party библиотек в духе numpy 
# обычно разделяют друг от друга пустой строкой

import numpy as np
# scipy -- надстройка над numpy, набор инструментов для научных вычислений
# Конкретно scipy.stats содержит большую часть нужных нам в курсе 
# распределений, статистических процедур в духе проверки гипотез etc.
import scipy.stats as sps

## Вместо предисловия

**Подумай, необходима ли оптимизация!**

1. На оптимизацию тратится время.
2. Скорее всего код станет непонятнее.
3. Не все оптимизации полезны. Оптимизируя по времени, вы можете увеличить расход памяти.

Перед оптимизацией стоит написать работающий код и тесты к нему.

## Профилирование

Профилирование - динамический анализ производительности кода.

Прежде чем приступать к оптимизации, нужно понять какой фрагмент кода нужно оптимизировать.

### Анализ времени работы

#### Явное профилирование строки / ячейки кода в Jupyter:

Иногда хочется измерить время исполнения участков кода целиком. При использовании IPython можно воспользоваться магическими функциями `%timeit` и `%%timeit`. Рассмотрим их на примере задачи "разворота" последовательности.

---

`%timeit` позволяет измерить время исполнения одной строки

In [ ]:
def slow_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    """
    :param seq: list
    :return: reversed list
    """
    reversed_seq = np.zeros(len(seq))
    for i in range(len(seq)):
        reversed_seq[i] = seq[len(seq) - i - 1]
    return reversed_seq

In [ ]:
# равновероятно выбрать 100 чисел от 0 до 100, т.е. сгенерировать 
# выборку размера 100 из дискретного равномерного распределения U[0, 100]
seq = sps.randint(0, 100).rvs(100)

Параметр `-n` позволяет указать, сколько раз запустить строчку с кодом. Многократный запуск нужен для того, чтобы получить более точную оценку времени работы (и обязателен для функций, использующих случайность в коде). 

In [ ]:
%timeit -n 5 slow_reverse(list(seq))
%timeit -n 5 seq[::-1]

5 loops, best of 5: 62.6 µs per loop
The slowest run took 50.62 times longer than the fastest. This could mean that an intermediate result is being cached.
5 loops, best of 5: 447 ns per loop


Команда `?timeit` позволяет посмотреть справку по этому magic-у.

In [ ]:
?timeit

`%%timeit` -- то же самое, но для целой ячейки с кодом.

In [ ]:
%%timeit 

seq = sps.randint(0, 100).rvs(100)
reversed_seq = np.zeros(len(seq))
for i in range(len(seq)):
    reversed_seq[i] = seq[len(seq) - i - 1]

1000 loops, best of 5: 736 µs per loop


Выводится среднее значение и среднеквадратичное отклонение.

---

Есть и более простой аналог, с однократным запуском:

In [ ]:
%%time

seq = sps.randint(0, 100).rvs(100)
reversed_seq = np.zeros(len(seq))
for i in range(len(seq)):
    reversed_seq[i] = seq[len(seq) - i - 1]

CPU times: user 1.61 ms, sys: 0 ns, total: 1.61 ms
Wall time: 1.6 ms


#### `line_profiler`

Позволяет собрать построчную аналитику для нескольких функций

In [ ]:
!pip install line_profiler
%load_ext line_profiler
%load_ext autoreload
%autoreload 4

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
%%writefile line_profiler_example.py

import typing as tp

import numpy as np
import scipy.stats as sps


def slow_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    """
    :param seq: list
    :return: reversed list
    """
    reversed_seq = np.zeros(len(seq))
    for i in range(len(seq)):
        reversed_seq[i] = seq[len(seq) - i - 1]
    return reversed_seq


def fast_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    return seq[::-1]


def profiler_example_wrapper(sequence_length: int = 100) -> None:
    """
    A statement to execute under the line-by-line profiler.
    :param length: length of the list to reverse
    """
    seq = sps.randint(0, 100).rvs(sequence_length)
    slow_reverse(seq)
    fast_reverse(seq)


Writing line_profiler_example.py


`line_profiler` запускается при помощи line magic-а `%lprun`. Обратите внимание, ему нужно **явно передать названия всех функций**, по которым вам нужна построчная статистика!

In [ ]:
import line_profiler_example

%lprun \
    -f line_profiler_example.slow_reverse \
    -f line_profiler_example.fast_reverse \
    line_profiler_example.profiler_example_wrapper()

In [ ]:
?lprun

Если вы работаете не в Jupyter-ноутбуке, то см. пример:

In [ ]:
%%writefile line_profiler_example.py

import typing as tp

import numpy as np
import scipy.stats as sps


# Навешиваем @profile на все функции, по которым нужна построчная статистика.
# Нигде его не определяем, просто навешиваем.

@profile
def slow_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    """
    :param seq: list
    :return: reversed list
    """
    reversed_seq = np.zeros(len(seq))
    for i in range(len(seq)):
        reversed_seq[i] = seq[len(seq) - i - 1]
    return reversed_seq


@profile
def fast_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    return seq[::-1]


if __name__ == '__main__':   
    seq = sps.randint(0, 100).rvs(100)
    slow_reverse(seq)
    fast_reverse(seq)


Overwriting line_profiler_example.py


In [ ]:
!python line_profiler_example.py

Traceback (most recent call last):
  File "line_profiler_example.py", line 11, in <module>
    @profile
NameError: name 'profile' is not defined


Далее вызываем утилиту `kernprof` (ставится сама вместе с `line_profiler)`. Флаг `-l` указывает ей собирать именно построчную статистику, флаг `-v` же указывает вывести отчёт в `stdout`

In [ ]:
!kernprof -l -v line_profiler_example.py

Wrote profile results to line_profiler_example.py.lprof
Timer unit: 1e-06 s

Total time: 0.000172 s
File: line_profiler_example.py
Function: slow_reverse at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           @profile
    12                                           def slow_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    13                                               """
    14                                               :param seq: list
    15                                               :return: reversed list
    16                                               """
    17         1          7.0      7.0      4.1      reversed_seq = np.zeros(len(seq))
    18       101         39.0      0.4     22.7      for i in range(len(seq)):
    19       100        125.0      1.2     72.7          reversed_seq[i] = seq[len(seq) - i - 1]
    20         1          1.0      1.0      0.6      return reversed_seq

Total ti

Ссылка на официальный репозиторий проекта: https://github.com/rkern/line_profiler

**Замечание:** Очевидно, что не хочется ломать работающий код добавлением декоратора `@profile`. В то же время, не хочется заново добавлять его каждый раз, когда понадобится запустить профайлер. 

**Решение** -- закомментировать все вхождения `@profile`. Это легко сделать в IDE и так же легко раскомментировать обратно. 

Если хочется быстро что-то проверить изнутри Jupyter-ноутбука, то можно обхитрить систему и в самом начале каждого модуля, где используется `@profile` добавить такой хак:

```python
try:
    profile  # throws an exception when profile isn't defined
except NameError:
    profile = lambda x: x   # if it's not defined simply ignore the decorator.
```

Это, очевидно, плохо в тех случаях, когда у вас в коде есть __другие__ сущности с названием `profile`, но как fast-and-dirty solution -- вполне себе вариант.

### Анализ количества потребляемой памяти

#### `memory_profiler`

Эта утилита аналогична `line_profiler` и позволяет измерить общее и построчное потребление памяти вашей программой.

In [ ]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30180 sha256=74c7a7bbf6eb532beaad5d3706be22f440da570803ac3c2a4e3976b8111b7266
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [ ]:
%load_ext memory_profiler

In [ ]:
def invoke_memory_hog() -> tp.List[tp.List[int]]:
    x = [[1] for _ in range(10 ** 4)]
    y = [[2] for _ in range(10 ** 6)]
    
    del x
    return y

Можно измерить общее потребление памяти (аналогично `%timeit`):

In [ ]:
%memit invoke_memory_hog()

peak memory: 285.04 MiB, increment: 97.07 MiB


`peak memory` - наибольшее значение расходуемой памяти системы во время работы программы. Нужно, чтобы посмотреть, насколько мы близки к тому, чтобы израсходовать всю RAM.

`increment` = `peak memory` - `starting memory`

In [ ]:
%memit?

Можно измерить потребление памяти по строкам. Есть утилита `%mprun`, но она не может работать с функциями, объявленными внутри юпитер-ноутбука, потому нужно навешивать декоратор `@profile`, как в предыдущем примере, и запускать через `python -m memory_profiler my_awesome_python_file.py`

In [ ]:
!python -m memory_profiler line_profiler_example.py

Filename: line_profiler_example.py

Line #    Mem usage    Increment  Occurences   Line Contents
    11   80.867 MiB   80.867 MiB           1   @profile
    12                                         def slow_reverse(seq: tp.List[tp.Any]) -> tp.List[tp.Any]:
    13                                             """
    14                                             :param seq: list
    15                                             :return: reversed list
    16                                             """
    17   80.867 MiB    0.000 MiB           1       reversed_seq = np.zeros(len(seq))
    18   80.867 MiB    0.000 MiB         101       for i in range(len(seq)):
    19   80.867 MiB    0.000 MiB         100           reversed_seq[i] = seq[len(seq) - i - 1]
    20   80.867 MiB    0.000 MiB           1       return reversed_seq


Filename: line_profiler_example.py

Line #    Mem usage    Increment  Occurences   Line Contents
    23   80.867 MiB   80.867 MiB           1   @profile
    24

Более содержательный пример:

In [ ]:
%%writefile memory_profiler_example.py

import numpy as np


@profile
def invoke_memory_hog():
    x = np.random.randint(10 ** 6)
    y = np.random.randn(10 ** 6)
    del x
    return y


if __name__ == '__main__':
    large_seq = invoke_memory_hog()
    print(len(large_seq))

Writing memory_profiler_example.py


In [ ]:
!python -m memory_profiler memory_profiler_example.py

1000000
Filename: memory_profiler_example.py

Line #    Mem usage    Increment  Occurences   Line Contents
     5   53.277 MiB   53.277 MiB           1   @profile
     6                                         def invoke_memory_hog():
     7   53.277 MiB    0.000 MiB           1       x = np.random.randint(10 ** 6)
     8   61.160 MiB    7.883 MiB           1       y = np.random.randn(10 ** 6)
     9   61.160 MiB    0.000 MiB           1       del x
    10   61.160 MiB    0.000 MiB           1       return y




Видно, что интерпретатор Python не такой уж глупый: он понимает, что переменная `x` никак содержательно не используется в коде, потому он просто вырезает все её вхождения в код. Память под неё даже не выделяется.